In [1]:
import FinanceDataReader as fdr
import asyncio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
class StkPrice:
    async def getPrice(self,code:str):
        res = fdr.DataReader(code)['Close']
        return res

In [2]:
code = ['005930','091170','003550','009200','261240','097950']
loader = StkPrice()
df = [await loader.getPrice(c) for c in code]


In [3]:
df = pd.DataFrame(df).T
df.columns = code

In [4]:
df = df.dropna()

In [5]:
rtn = ((df.shift(30) - df)/df).dropna()

In [6]:
mn = rtn.mean(axis=0)*100

In [7]:
stdev =  rtn.std(axis=0)*100

In [8]:
rtn_cov = rtn.cov()

In [9]:
from scipy.optimize import minimize

def objfunc(w,rtn_cov):
    return np.sqrt(w.T@rtn_cov@w)

weights = np.array([1.0/6] * 6)
bound = [(0,1) for i in range(6)]
params = ({ 'type': 'eq', 'fun': lambda x: np.sum(x) - 1 })
res = minimize(objfunc,weights,(rtn_cov),method='SLSQP',bounds=bound,constraints = params)

In [10]:
res.x

array([0.09028755, 0.08067513, 0.0220514 , 0.        , 0.79469889,
       0.01228703])

In [11]:
rtn_cov

,005930,091170,003550,009200,261240,097950
005930,0.007265,0.003746,0.004088,0.005080,-0.001013,0.003082
091170,0.003746,0.007814,0.003831,0.005464,-0.001005,0.002631
003550,0.004088,0.003831,0.010900,0.004087,-0.000874,0.005081
009200,0.005080,0.005464,0.004087,0.015814,-0.000937,0.004559
261240,-0.001013,-0.001005,-0.000874,-0.000937,0.000607,-0.000537
097950,0.003082,0.002631,0.005081,0.004559,-0.000537,0.008932
